# Experiment tracking with MLflow

* Add parameter tuning to the notebook
* Show how it looks in MLflow
* Select the best one
* Autolog

Input:
* Notebook from previous unit

Output:
* Updated notebook
* More logging in MLflow

## Setup

In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge

from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

In [2]:
%env MLFLOW_TRACKING_URI=http://localhost:5000
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ACCESS_KEY_ID=2vSrPs21nZYaUQvovgRL
%env AWS_SECRET_ACCESS_KEY=yCqF29KU1qbykEnsceWMDRNvPelgGAVBmyD6PeU5

env: MLFLOW_TRACKING_URI=http://localhost:5000
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ACCESS_KEY_ID=2vSrPs21nZYaUQvovgRL
env: AWS_SECRET_ACCESS_KEY=yCqF29KU1qbykEnsceWMDRNvPelgGAVBmyD6PeU5


In [3]:
import os
import mlflow

assert "MLFLOW_TRACKING_URI" in os.environ
assert "MLFLOW_S3_ENDPOINT_URL" in os.environ
assert "AWS_ACCESS_KEY_ID" in os.environ
assert "AWS_SECRET_ACCESS_KEY" in os.environ

# mlflow.set_tracking_uri("http://localhost:5000/")
mlflow.set_experiment("nyc-taxi")

<Experiment: artifact_location='s3://mlflow/1', creation_time=1700794101778, experiment_id='1', last_update_time=1700794101778, lifecycle_stage='active', name='nyc-taxi', tags={}>

## Data Preparation

In [4]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
train_data_path = "./data/green_tripdata_2021-01.parquet"
val_data_path = "./data/green_tripdata_2021-02.parquet"

df_train = read_dataframe(train_data_path)
df_val = read_dataframe(val_data_path)
len(df_train), len(df_val)

(73908, 61921)

In [6]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

## Model Training

In [9]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [10]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.set_tag("developer", "sugab")

        mlflow.log_params(params)

        model = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=500,
            evals=[(valid, "validation")],
            early_stopping_rounds=25
        )

        y_pred = model.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {"loss": rmse, "status": STATUS_OK}

In [ ]:
search_space = {
    "max_depth": scope.int(hp.quniform("max_depth", 4, 100, 1)),
    "learning_rate": hp.loguniform("learning_rate", -3, 0),
    "reg_alpha": hp.loguniform("reg_alpha", -5, -1),
    "reg_lambda": hp.loguniform("reg_lambda", -6, -1),
    "min_child_weight": hp.loguniform("min_child_weight", -1, 3),
    "objective": "reg:linear",
    # "device": "cuda",
    "seed": 42,
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=25,
    trials=Trials()
)

### Fit best params to the model

In [11]:
best_params = {
    "learning_rate": 0.2526805505341685,
    "max_depth": 21,
    "min_child_weight": 1.4699844585464745,
    "objective": "reg:linear",
    "reg_alpha": 0.03181433933064131,
    "reg_lambda": 0.0050476303572755754,
    "seed": 42,
}

mlflow.xgboost.autolog()

best_model = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=20,
    evals=[(valid, "validation")],
    early_stopping_rounds=25,
)

2023/11/24 16:51:35 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.
2023/11/24 16:51:35 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '92f667d0a3d34385b96d400bc7760682', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:10.26015
[1]	validation-rmse:8.94980
[2]	validation-rmse:8.10100
[3]	validation-rmse:7.56022
[4]	validation-rmse:7.21732
[5]	validation-rmse:7.00285
[6]	validation-rmse:6.86549
[7]	validation-rmse:6.77423
[8]	validation-rmse:6.71627
[9]	validation-rmse:6.67279
[10]	validation-rmse:6.64493
[11]	validation-rmse:6.62168
[12]	validation-rmse:6.60743
[13]	validation-rmse:6.59441
[14]	validation-rmse:6.58689
[15]	validation-rmse:6.58038
[16]	validation-rmse:6.57834
[17]	validation-rmse:6.57420
[18]	validation-rmse:6.57302
[19]	validation-rmse:6.57114


2023/11/24 16:52:29 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.


## What i learned

* Experiment tracking meggunakan MLflow, caranya dengan membuat scope with mlflow.start_run() kemudian masukan kode trainer di dalamnya
* Implementasi dari experiment tracking ini adalah kita bisa menggunakannya sebagai logging ketika hyperparameter tuning, dalam hal ini menggunakan library `hyperopt`
* Setelah menemukan hyperparameter terbaik, kita dapat melakukan trainingnya lagi pada model terbaru dengan hyperparameter tersebut
* Selain itu, kita bisa melakukan logging secara otomatis berdasarkan library yang dipakai menggunakan autolog
* Yang belum dibahas di notebook ini adalah **cara menyimpan model yang telah ditraining** (menyimpan artifact)

## Reference

* [MLOps Zoomcamp 2.3 - Experiment tracking with MLflow](https://www.youtube.com/watch?v=iaJz-T7VWec&list=PL3MmuxUbc_hIUISrluw_A7wDSmfOhErJK&index=11)